<a href="https://colab.research.google.com/github/khanabdulmajid/tensorflow_deep_learning/blob/main/05_Transfer_Learning_in_TensorFlow_Part_2_Fine_tuning_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🛠 05. Transfer Learning in TensorFlow Part 2: Fine-tuning Exercises
1. Use feature-extraction to train a transfer learning model on 10% of the Food Vision data for 10 epochs using `tf.keras.applications.EfficientNetB0` as the base model. Use the ModelCheckpoint callback to save the weights to file.
2. Fine-tune the last 20 layers of the base model you trained in 2 for another 10 epochs. How did it go?
3. Fine-tune the last 30 layers of the base model you trained in 2 for another 10 epochs. How did it go?
4. Write a function to visualize an image from any dataset (train or test file) and any class (e.g. "steak", "pizza"... etc), visualize it and make a prediction on it using a trained model.
📖
# 05. Transfer Learning in TensorFlow Part 2: Fine-tuning Extra-curriculum
* Read the documentation on data augmentation in TensorFlow.
* Read the ULMFit paper (technical) for an introduction to the concept of freezing and unfreezing different layers.
* Read up on learning rate scheduling (there's a TensorFlow callback for this), how could this influence our model training?
> * If you're training for longer, you probably want to reduce the learning rate as you go... the closer you get to the bottom of the hill, the smaller steps you want to take. Imagine it like finding a coin at the bottom of your couch. In the beginning your arm movements are going to be large and the closer you get, the smaller your movements become.




In [ ]:
# Get helper_functions.py script from course GitHub
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions we're going to use
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

--2023-08-28 11:01:07--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-08-28 11:01:07 (91.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Get 10% of the data of the 10 classes
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2023-08-28 11:01:12--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   278MB/s    in 0.6s    

2023-08-28 11:01:12 (278 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### 1. Use feature-extraction to train a transfer learning model on 10% of the Food Vision data for 10 epochs using `tf.keras.applications.EfficientNetB0` as the base model. Use the ModelCheckpoint callback to save the weights to file.

In [ ]:
checkpoint_path = "food_vision_checkpoint/"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True)

base_model= tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
for i,layers in enumerate(base_model.layers):
  print(i,layers.name)

In [ ]:
train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"

In [ ]:
IMG_SIZE=(224,224)
train_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    label_mode="categorical",
    image_size=IMG_SIZE
)
test_data=tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    label_mode="categorical",
    image_size=IMG_SIZE
)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
inputs= tf.keras.layers.Input(shape=(224,224,3),name="inputLayer")
data_augmentation=tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomHeight(0.2),
  tf.keras.layers.RandomWidth(0.2),
    ],
    name="data_augmentation_layer"
    )

x = data_augmentation (inputs)

x=base_model(x,training=False)

x = tf.keras.layers.GlobalAveragePooling2D(name="GlobalAveragePooling")(x)
output = tf.keras.layers.Dense(10,activation="softmax",name="output_Layer")(x)
model_0 = tf.keras.Model(inputs,output)



In [ ]:
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"]
                )

history_10_percent=model_0.fit(
    train_data_10_percent,
    epochs=10,
    validation_data=test_data,
    validation_steps=int(0.25*len(test_data)),
    callbacks=[checkpoint_callback]
)

Epoch 1/10
24/24 [==============================] - 151s 6s/step - loss: 1.9817 - accuracy: 0.3400 - val_loss: 1.4262 - val_accuracy: 0.6513
Epoch 2/10
24/24 [==============================] - 142s 6s/step - loss: 1.3038 - accuracy: 0.6920 - val_loss: 0.9576 - val_accuracy: 0.7928
Epoch 3/10
24/24 [==============================] - 143s 6s/step - loss: 0.9897 - accuracy: 0.7600 - val_loss: 0.7609 - val_accuracy: 0.8273
Epoch 4/10
24/24 [==============================] - 144s 6s/step - loss: 0.8325 - accuracy: 0.7893 - val_loss: 0.6614 - val_accuracy: 0.8470
Epoch 5/10
24/24 [==============================] - 146s 6s/step - loss: 0.7322 - accuracy: 0.8147 - val_loss: 0.5973 - val_accuracy: 0.8734
Epoch 6/10
24/24 [==============================] - 150s 6s/step - loss: 0.6595 - accuracy: 0.8480 - val_loss: 0.5624 - val_accuracy: 0.8618
Epoch 7/10
24/24 [==============================] - 131s 6s/step - loss: 0.5808 - accuracy: 0.8707 - val_loss: 0.5521 - val_accuracy: 0.8520
Epoch 8/10
24

### 2. Fine-tune the last 20 layers of the base model you trained in 2 for another 10 epochs. How did it go?

In [ ]:
model_0.layers[2].trainable=True
for layer in model_0.layers[2].layers[:-20]:
  layer.trainable=False


In [ ]:
for layer in model_0.layers[2].layers:
  print(layer.name, layer.trainable)



input_1 False
rescaling False
normalization False
rescaling_1 False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_dwconv_pad False
block2a_dwconv False
block2a_bn False
block2a_activation False
block2a_se_squeeze False
block2a_se_reshape False
block2a_se_reduce False
block2a_se_expand False
block2a_se_excite False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_dwconv False
block2b_bn False
block2b_activation False
block2b_se_squeeze False
block2b_se_reshape False
block2b_se_reduce False
block2b_se_expand False
block2b_se_excite False
block2b_proj

In [ ]:
history_10_percent.epoch[-1]

9

In [ ]:
initial_epochs=10
fine_tuning_epochs =initial_epochs+10
model_0.layers[2].trainable=True
for layer in model_0.layers[2].layers[:-20]:
  layer.trainable=False

model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"]
                )

history_10_percent_10_trainable_layers=model_0.fit(
    train_data_10_percent,
    epochs=fine_tuning_epochs,
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[checkpoint_callback],
    initial_epoch=history_10_percent.epoch[-1]
)

Epoch 10/20
24/24 [==============================] - 282s 12s/step - loss: 0.6593 - accuracy: 0.7947 - val_loss: 0.4735 - val_accuracy: 0.8540
Epoch 11/20
24/24 [==============================] - 268s 12s/step - loss: 0.2787 - accuracy: 0.9147 - val_loss: 0.6688 - val_accuracy: 0.8168
Epoch 12/20
24/24 [==============================] - 279s 12s/step - loss: 0.1887 - accuracy: 0.9387 - val_loss: 0.4183 - val_accuracy: 0.8792


### 3. Fine-tune the last 30 layers of the base model you trained in 2 for another 10 epochs. How did it go?

In [ ]:
# Loading the weights we've saved before in the food vision checkpoint
model_0.load_weights('food_vision_checkpoint/')

In [ ]:
base_model.trainable = True

# Refreeze every layer except for the last 30
for layer in base_model.layers[:-30]:
  layer.trainable = False

In [ ]:
for num , layer in enumerate(model_0.layers[2].layers):
  print(num,layer.name, layer.trainable)

In [ ]:
model_0.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics = ['accuracy'])

In [ ]:
fine_tune_epochs = initial_epochs + 10
history_fine_tune_model_30 = model_0.fit(train_data_10_percent,
                                      epochs = fine_tune_epochs ,
                                      steps_per_epoch=len(train_data_10_percent),
                                      validation_data = test_data ,
                                      validation_steps = len(test_data),
                                      initial_epoch =history_10_percent.epoch[-1])

 ## 4. Write a function to visualize an image from any dataset (train or test file) and any class (e.g. "steak", "pizza"... etc), visualize it and make a prediction on it using a trained model.